In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [4]:
filename = "models/phi-3-16.gguf"
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(filename, device_map="auto")

In [6]:
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
from transformers import AutoTokenizer
import jax
# tokenizer = load_tokenizer(filename)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
# prompt = """<s><|user|>How to explain Internet for a medieval knight?<|end|><|assistant|>It"""
# prompt = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>Hi,"""
prompt = "<s><|system|>You are an assistant.</s><|user|>Hello!</s><|assistant|>Hi,"
# prompt = "<|user|>"
tokens = (tokenizer.encode(prompt) * 40)[:4096]
token_array = jnp.asarray(tokens).reshape((1, -1))
token_array = jnp.repeat(token_array, 16, axis=0)
token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")
inputs = llama.inputs.from_basic_segments(token_array)
# token_array = pz.nx.nmap(lambda x: x.T)(token_array.untag("seq", "batch")).tag("batch", "seq")
# token_array.data_array.shape

[autoreload of absl.flags._argument_parser failed: Traceback (most recent call last):
  File "/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/

In [11]:
from micrlhf.llama import LlamaBlock
from micrlhf.flash import flashify
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)

In [12]:
jax.block_until_ready(get_resids_call(inputs))

(<NamedArray bfloat16(| batch:16, seq:640, vocabulary:32064) ≈-0.15 ±5.1 [≥-6.5e+01, ≤3.8e+01] zero:604_160 nonzero:327_731_200 (wrapping jax.Array)>,
 [SideOutputValue(
    keypath=(
      GetAttrKey(name='body'),
      GetAttrKey(name='body'),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=1),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=0),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=0),
      GetAttrKey(name='side_out'),
    ),
    tag='resid_pre_0',
    value=<NamedArray bfloat16(| embedding:3072, batch:16, seq:640) ≈-8.9e-05 ±0.026 [≥-0.21, ≤0.22] nonzero:31_457_280 (wrapping jax.Array)>,
  ),
  SideOutputValue(
    keypath=(
      GetAttrKey(name='body'),
      GetAttrKey(name='body'),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=1),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=1),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=0),
      GetAttrKey(name='side_out'),
    ),
    tag='resid_pre_1',
    value=<NamedArray bfloat16(| batch:16, seq:640, embedding:3072) ≈-0.00062 ±0.065 [≥-4.8, ≤3.3] nonzero:31_457_280 (wrapping jax.Array)>,
  ),
  SideOutputValue(
    keypath=(
      GetAttrKey(name='body'),
      GetAttrKey(name='body'),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=1),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=2),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=0),
      GetAttrKey(name='side_out'),
    ),
    tag='resid_pre_2',
    value=<NamedArray bfloat16(| batch:16, seq:640, embedding:3072) ≈-0.00064 ±0.092 [≥-7.2, ≤5.7] nonzero:31_457_280 (wrapping jax.Array)>,
  ),
  SideOutputValue(
    keypath=(
      GetAttrKey(name='body'),
      GetAttrKey(name='body'),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=1),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=3),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=0),
      GetAttrKey(name='side_out'),
    ),
    tag='resid_pre_3',
    value=<NamedArray bfloat16(| batch:16, seq:640, embedding:3072) ≈-0.0015 ±1.8 [≥-1e+03, ≤8.7e+02] nonzero:31_457_280 (wrapping jax.Array)>,
  ),
  SideOutputValue(
    keypath=(
      GetAttrKey(name='body'),
      GetAttrKey(name='body'),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=1),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=4),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=0),
      GetAttrKey(name='side_out'),
    ),
    tag='resid_pre_4',
    value=<NamedArray bfloat16(| batch:16, seq:640, embedding:3072) ≈-0.0012 ±1.8 [≥-1e+03, ≤8.7e+02] nonzero:31_457_280 (wrapping jax.Array)>,
  ),
  SideOutputValue(
    keypath=(
      GetAttrKey(name='body'),
      GetAttrKey(name='body'),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=1),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=5),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=0),
      GetAttrKey(name='side_out'),
    ),
    tag='resid_pre_5',
    value=<NamedArray bfloat16(| batch:16, seq:640, embedding:3072) ≈-0.0033 ±3.4 [≥-2e+03, ≤1.5e+03] nonzero:31_457_280 (wrapping jax.Array)>,
  ),
  SideOutputValue(
    keypath=(
      GetAttrKey(name='body'),
      GetAttrKey(name='body'),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=1),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=6),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=0),
      GetAttrKey(name='side_out'),
    ),
    tag='resid_pre_6',
    value=<NamedArray bfloat16(| batch:16, seq:640, embedding:3072) ≈-0.004 ±3.4 [≥-2e+03, ≤1.5e+03] nonzero:31_457_280 (wrapping jax.Array)>,
  ),
  SideOutputValue(
    keypath=(
      GetAttrKey(name='body'),
      GetAttrKey(name='body'),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=1),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=7),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=0),
      GetAttrKey(name='side_out'),
    ),
    tag='resid_pre_7',
    value=<NamedArray bfloat16(| batch:16, seq